# HW2 Overview

In this assignment, we will study language model. You will get the basic ideas of maximum likelihood estimation, smoothing, generate text documents from language models, and language model evaluation. 

We will reuse the same Yelp dataset and refer to each individual user review as a **document** (e.g., as in computing document frequency). You should reuse your JSON parser in this assignment.

The same pre-processing steps you have developed in HW1 will be used in this assignment, i.e., tokenization, stemming and normalization. Note: **NO** stopword removal is needed in this assignment. 



# Statistical Language Models

### 1. Maximum likelihood estimation for statistical language models with proper smoothing (50pts)

Use all the review documents to estimate a unigram language model $p(w)$ and two bigram language models (with different smoothing methods specified below). Note those language models are corpus-level models, i.e., aggregating all the words across different documents.

When estimating the bigram language models, using linear interpolation smoothing and absolute discount smoothing based on the unigram language model $p_u(w)$ to get two different bigram language models accordingly, i.e., $p^L(w_i|w_{i-1})$ and $p^A(w_i|w_{i-1})$. In linear interpolation smoothing, set the parameter $\lambda=0.9$; and in absolute discount smoothing, set the parameter $\delta=0.1$.

Specifically, when estimating $p^L(w_i|w_{i-1})$ and $p^A(w_i|w_{i-1})$, you should use the unigram language model $p(w_i)$ as the reference language model in smoothing. For example, in linear interpolation smoothing, the resulting smoothing formula looks like this,

$$p^L(w_i|w_{i-1})=(1-\lambda) \frac{c(w_{i-1}w_i)}{c(w_{i-1})} + \lambda p(w_i)$$ 
where $c(w_{i-1}w_i)$ is the frequency of bigram $w_{i-1}w_i$ in the whole corpus.

From the resulting two bigram language models, find the top 10 words that are most likely to follow the word "good", i.e., rank the words in a descending order by $p^L(w|good")$ and $p^A(w|good")$ and output the top 10 words. Are those top 10 words the same from these two bigram language models? Explain your observation.

*HINT: to reduce space complexity, you do not need to actually maintain a $V\times V$ array to store the counts and probabilities for the bigram language models. You can use a sparse data structure, e.g., hash map, to store the seen words/bigrams, and perform the smoothing on the fly, i.e., evoke some function calls to return the value of $p^L(w|good")$ and $p^A(w|good")$.* 

**What to submit**:

1. Paste your implementation of the linear interpolation smoothing and absolute discount smoothing.
2. The top 10 words selected from the corresponding two bigram language models.
3. Your explanation of the observations about the top words under those two bigram language models.



In [1]:
from utils import *
import pickle

In [2]:
#unigram language model p(w)
list_of_tokenized_reviews = []
with open('list_of_tokenized_reviews.pickle', 'rb') as file:
    list_of_tokenized_reviews = pickle.load(file)
token_freq_dict = total_term_frequency(list_of_tokenized_reviews)
total_num_tokens = sum([len(tokens) for tokens in list_of_tokenized_reviews])
token_prob_dict = {}
for key in token_freq_dict.keys():
    token_prob_dict[key] = token_freq_dict[key] / total_num_tokens


In [ ]:
#bigram language model with linear interpolation smoothing
all_bigrams = get_all_bigrams(list_of_tokenized_reviews)
bigram_freq_dict = total_bigram_freqency(all_bigrams)
def p_linear(first, second):
    lam = 0.9
    try:
        prob = (1-lam) * (bigram_freq_dict[(first, second)] / token_freq_dict[first]) + (lam * token_prob_dict[second])
    except:
        prob = (lam * token_prob_dict[second])
    return prob
#get the top most likely words to follow "good"
linear_probabilities = {}
for token in token_prob_dict.keys():
    linear_probabilities[token] = p_linear('good', token)
ans = dict(sorted(linear_probabilities.items(), key = lambda x: x[1], reverse = True)[:10])
for line in ans:
    print(f'{line} : {ans[line]}')    

In [ ]:
target_bigrams = {}
for bigram in bigram_freq_dict.keys():
    if bigram[0] == 'good':
        target_bigrams[bigram] = bigram_freq_dict[bigram]
ans = dict(sorted(target_bigrams.items(), key = lambda x: x[1], reverse = True)[:10])
for line in ans:
    print(f'{line} : {ans[line]}')

In [ ]:
#bigram language model with absolute discount smoothing
def p_abs_disc(first, second):
    delta = 0.1
    d_u = 0
    prob = (max(bigram_freq_dict[(first, second)] - delta, 0) + (delta * d_u * token_prob_dict[second])) / (token_freq_dict[first])

### 2. Generate text documents from a language model (40pts)

Fixing the document length to 20, generate 10 documents by sampling words from $p(w)$, $p^L(w_i|w_{i-1})$ and $p^A(w_i|w_{i-1})$ respectively.

*HINT: you can use $p(w)$ to generate the first word of a document and then sampling from the corresponding bigram language model when generating from $p^L(w_i|w_{i-1})$ and $p^A(w_i|w_{i-1})$.* 

**What to submit**:

1. Paste your implementation of the sampling procedure from a language model.
2. The 10 documents generated from $p(w)$, $p^L(w_i|w_{i-1})$ and $p^A(w_i|w_{i-1})$ accordingly, and the corresponding likelihood given by the used language model.

# Reading Assignment — Belief or Bias in Information Retrieval (10pts)
In our class, we have learned both classical and modern information retrieval evaluation methods. And their shared goal is to assess if a retrieval system can satisfy users' information need. Such an evaluation directly leads to the subsequent optimization of retrieval system, e.g., optimize the ranking for click-through rates. But should a system please its users so as to improve the metrics or should it educate the users about what is right and wrong?

Let's read the paper ["Beliefs and biases in web search"](https://dl.acm.org/doi/10.1145/2484028.2484053), which is the best paper in SIGIR'2013. Based on the findings of this paper and current public concern/debate of the wide spread of misinformation on the web, what kind of suggestion do you want to give to Google and Bing to improve the situation? You can focus on the search evaluation, document retrieval and ranking, or any aspect related to the retrieval process.

# Extra Credits (5pts)

You are encouraged to further investigate the relation between classic language model and the trending Large Language Models. How LLMs differ from unigram and bigram models we implemented? It is okay to consult LLMs for this question :\) 

# Submission

This assignment has in total 100 points. The deadline is Feb 20 23:59 PDT. You should submit your report in **PDF** using the homework latex template, and submit your code (notebook)。